# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv


#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])




# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        
        
    

8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [4]:
data1 = pd.read_csv('event_datafile_new.csv')
data1.head(1)


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now create tables to run the following queries. With Apache Cassandra, need to model the database tables on the queries want to run.

In [9]:
query = "DROP TABLE IF EXISTS song_by_user_session"
try:
    session.execute(query)
except Exception as e:
    print(e)


For table song_by_user_session, the column sessionId and itemInSession were used as partition key because the queries will filter by the two columns.

In [10]:
query = "CREATE TABLE IF NOT EXISTS song_by_user_session "
query = query + "(sessionId int, itemInSession int, artist text, firstName text, gender text, \
lastName text, length float, level text, \
location text, song text, userId int, \
PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "select artist, song, length from song_by_user_session where sessionId = 338 and itemInSession = 4"


In [12]:
# Set up the CSV file with Apache Cassandra#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user_session (sessionId, itemInSession, artist, firstName, gender, \
        lastName, length, level, location, song, userId )"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9], int(line[10])))
        
      

In [13]:
## Use the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [14]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "DROP TABLE IF EXISTS artist_by_user_session"
try:
    session.execute(query)
except Exception as e:
    print(e)
  

For table song_by_user_session2, the columns userId and sessionId were used as partition key because the queries will filter by the two columns. Column itemInSession was used as clustering column to help make up a unique key.

In [15]:
query = "CREATE TABLE IF NOT EXISTS artist_by_user_session"
query = query + "(userId int, sessionId int, itemInSession int, \
song text, artist text, firstname text, lastname text, \
Primary Key((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [16]:
query2 = "select song,artist,firstname,lastname from artist_by_user_session where userId = 10 and sessionId = 182"


In [17]:
# Set up the CSV file with Apache Cassandra#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_by_user_session (userId, sessionId, itemInSession, song, \
        artist, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[9],line[0],line[1],line[2]))
    
      

In [18]:
## Use the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song, row.artist, row.firstname, row.lastname)

Keep On Keepin' On Down To The Bone Sylvie F
Greece 2000 Three Drives Sylvie F
Kilometer Sebastien Tellier Sylvie F
Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Lonnie Gordon Sylvie F


In [19]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "DROP TABLE IF EXISTS users_for_song"
try:
    session.execute(query)
except Exception as e:
    print(e)


For table song_by_user_session3, the column song was used as a partition key because the queries will filter by this column. Column userId was used as clustering column to help make up a unique key.

In [20]:
query = "CREATE TABLE IF NOT EXISTS users_for_song"
query = query + "(song text, userId int, artist text, \
firstname text, lastname text, \
Primary Key(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [21]:
query3 = "select firstname, lastname from users_for_song where song='All Hands Against His Own'"


In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_for_song (song, userid, artist, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[9],int(line[10]),line[0],line[1],line[4]))
    

In [23]:
## Use the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [24]:
## Drop the table before closing out the sessions
query = "DROP TABLE song_by_user_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE artist_by_user_session"
try:
    session.execute(query)
except Exception as e:
    print(e)
   
query = "DROP TABLE users_for_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()